In [1]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.utilities import GoogleSearchAPIWrapper
from langchain.utilities import WikipediaAPIWrapper
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import StringPromptTemplate
from langchain import OpenAI, SerpAPIWrapper, LLMChain
from typing import List, Union, Any, Optional, Type
from langchain.schema import AgentAction, AgentFinish
import re
from langchain import PromptTemplate
from langchain.tools import BaseTool
from langchain.callbacks.manager import AsyncCallbackManagerForToolRun, CallbackManagerForToolRun
from langchain.utilities import GoogleSerperAPIWrapper

from llm_client import AlpacaLLM
from typing import Callable
import json
import requests
from langchain.tools import BaseTool, StructuredTool, tool
import random
from langchain.memory import ConversationBufferMemory

In [2]:
import os
os.environ["SERPER_API_KEY"] = 'f90fe84e78ef9d2d8e377ab5c6fe3a4a25f42ef0'
os.environ["LANGCHAIN_TRACING_V2"] = 'true'
os.environ["LANGCHAIN_ENDPOINT"] = 'https://api.smith.langchain.com'
os.environ["LANGCHAIN_API_KEY"] = 'ls__413a7563fa034592be6c6a241176932a'
os.environ["LANGCHAIN_PROJECT"] = 'LLM Patient Monitoring'

In [3]:
@tool
def send_api_to_medic(query: str) -> str: 
    """Function that send a message to the medic if the patient has a symtomps that he tells to the chatbot, and only run it once.
    The query is the message that the patient send to the chatbot."""
    url = "http://127.0.0.1:8000/get_message/"
    # message = "Hello its send api"
    user_id = str(random.randint(1, 100))

    data = {
        "message": query,
        "user_id": user_id
    }

    #Turn data into json for the request
    data = json.dumps(data)

    response = requests.post(url, data=data)
    return ' Success sending message. Please provide search query to get the result.'

@tool
def search(query: str) -> str: 
    """Function that it use when you searching up something on google"""
    return GoogleSerperAPIWrapper().run(query)

In [4]:
tools_list = [send_api_to_medic, search]
tool_names = [i.name for i in tools_list]

In [5]:
template = """You're name is PANDA, an assitant for patient hemodialysis. Your job to be a companion and assist the patient or user for their questions. You must gives helpful, detailed, and polite answers to the user's questions.

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Answer the following questions as best you can. You have access to the following tools, and user it based on the context of the questions:

{tools}

**Strictly** use the following format:

Question: the input question you must answer
Thought: you should always think about what to do, in every step after Observation
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action, should be a question.
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

For examples of how to use the tools, see the examples below.:
Question: I have a really bad headache?
Thought: First, I need send a message to the medic.
Action: send_api_to_medic
Action Input: "I have a really bad headache?"
Observation: Success sending message. Please provide search query to get the result.
Thought: Now, I should start the search.
Action: search
Action Input: What causes a really bad headache?
Observation: Bad headache can be caused by a number of things, including dehydration, stress, and tension.
Thought: I now know the final answer.
Final Answer: Anupama Nadella is 50 years old.

Sometimes user's questions can be answered directly without using any tools.
For example:
Question: Hello, how are you?
Thought: I can answer this question directly.
Final Answer: I am doing well, thank you for asking.

If the questions does not require any tools, you can answer directly without using any tools, and to the Final Answer directly.

Begin the task by answering the following question, **remember** to follow the format above.:

### Input:
{input}

### Response:
{agent_scratchpad}
"""

In [6]:
class CustomPromptTemplate(StringPromptTemplate):
    """Schema to represent a prompt for an LLM.

    Example:
        .. code-block:: python

            from langchain import PromptTemplate
            prompt = PromptTemplate(input_variables=["foo"], template="Say {foo}")
    """

    input_variables: List[str]
    """A list of the names of the variables the prompt template expects."""

    template: str
    """The prompt template."""

    template_format: str = "f-string"
    """The format of the prompt template. Options are: 'f-string', 'jinja2'."""

    validate_template: bool = False
    """Whether or not to try validating the template."""
    
    tools_getter: List[StructuredTool]

    def format(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}"
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        ############## NEW ######################
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join(
            [f"{tool.name}: {tool.description}" for tool in self.tools_getter]
        )
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools_getter])
        return self.template.format(**kwargs)

In [7]:
prompt = CustomPromptTemplate(input_variables=["input", "intermediate_steps"],
                              template=template,validate_template=False, tools_getter=tools_list)

In [8]:
class CustomOutputParser(AgentOutputParser):
    
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        
        match = re.search(regex, llm_output, re.DOTALL)
        
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        if match:

        
            action = match.group(1).strip() #Action send_api
            action_input = match.group(2) #Action Input "What is the weather today?"
            # Return the action and action input
            return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)
        
output_parser = CustomOutputParser()

In [9]:
llm = AlpacaLLM()

In [10]:
llm_chain = LLMChain(llm=llm, prompt=prompt)

agent = LLMSingleActionAgent(
    llm_chain=llm_chain, 
    output_parser=output_parser,
    stop=["\nObservation:"],
    allowed_tools=tool_names
)
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools_list, verbose=True)

c:\Users\Zain\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain.agents.agent.LLMSingleActionAgent` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [11]:
agent_executor.run("Hello, who are you?")

c:\Users\Zain\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
Question: Hello, who are you?
Thought: I can answer this question directly.
Final Answer: My name is Panda, your assistant for patient hemodialysis.

> Finished chain.


'My name is Panda, your assistant for patient hemodialysis.'